In [2]:
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import qml_interface as qmi

In [3]:
# load dE at 20 and 30 Ang for both molecules
dsdg_003664_lamc = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003664/atomic_energies_with_mic.txt')
dsdg_003664_lamf = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/number_lambda_points/dsgdb9nsd_003664_atomic_energies_fine_grid.txt')

dsdg_003700_lamc = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003700/atomic_energies_with_mic.txt')
dsdg_003700_lamf = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/number_lambda_points/dsgdb9nsd_003700_atomic_energies_fine_grid.txt')

In [5]:
# find the most similar atoms

# make coulomb representations
reps_dsdg_003664 = qmi.generate_atomic_representations([dsdg_003664_lamc], [len(dsdg_003664_lamc)])
reps_dsdg_003700 = qmi.generate_atomic_representations([dsdg_003700_lamc], [len(dsdg_003700_lamc)])

# calculate distance matrix

dmatrix = np.zeros((len(reps_dsdg_003664), len(reps_dsdg_003700)))
              
for i, r in enumerate(reps_dsdg_003664):
    for j, r2 in enumerate(reps_dsdg_003700):
        dmatrix[i,j] = np.linalg.norm(r-r2)

# find closest pairs
pairs = []

for i in range(dmatrix.shape[0]):
    dmatrix_row = dmatrix[i]
    ind = np.where( dmatrix_row == np.amin(dmatrix_row[np.where(dmatrix_row > 0)]))
    tmp = [i, ind[0][0]]
    #tmp.sort()
    pairs.append(tmp)
# pairs = []
# import itertools
# pairs.sort()
# pairs = list(pairs for pairs,_ in itertools.groupby(pairs))

In [6]:
# calculate the energy difference between these atoms at boxsize = 20 and boxsize = 30
def delta_E(comp1, comp2, pairs):
    mu = []
    e = []
    e_at = []
    for p in pairs:
        mu.append(comp1[p[0],4] - comp2[p[1],4])
        e.append(comp1[p[0],5] - comp2[p[1],5])
        e_at.append(comp1[p[0],6] - comp2[p[1],6])
    return(np.array(mu), np.array(e), np.array(e_at))

In [8]:
# coarse grid (6 points)
muc, ec, e_atc = delta_E(dsdg_003664_lamc, dsdg_003700_lamc, pairs)
# fine grid
muf, ef, e_atf = delta_E(dsdg_003664_lamf, dsdg_003700_lamf, pairs)

In [13]:
muf

array([ 0.01088297,  0.00416716,  0.00052097, -0.00980282, -0.00490456,
        0.19402489, -0.16603476,  0.08836453, -0.08857384,  0.00466438,
        0.18121869, -0.00910352, -0.18504775])

In [15]:
(muc-muf)

array([-2.64355504e-03, -4.77771114e-03, -9.86422750e-04, -4.73773321e-03,
        2.07287613e-03,  7.82657108e-04,  5.68988453e-04,  1.10360110e-04,
       -6.34543698e-05,  7.83135111e-04, -1.00383917e-02,  1.02229805e-02,
        2.67390550e-03])

In [10]:
ec-ef

array([-1.58613302e-02, -2.86662668e-02, -5.91853650e-03, -3.79018657e-02,
        1.24372568e-02,  5.47859976e-03,  3.98291917e-03,  1.10360110e-04,
       -6.34543698e-05,  7.83135111e-04, -1.00383917e-02,  1.02229805e-02,
        2.67390550e-03])

In [11]:
e_atc - e_atf

array([-0.01103358, -0.02383852, -0.00109079, -0.03307412,  0.017265  ,
        0.01030635,  0.00881066,  0.00493811,  0.00476429,  0.00561088,
       -0.00521065,  0.01505073,  0.00750165])